<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/seirs_comparacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp
import time

# Modelo SEIRS (Equações de Diferenças)
def seirs_disc(S0, E0, I0, R0, beta, sigma, gamma, omega, N, dt, T):
    passos = int(T / dt)
    S, E, I, R = [S0], [E0], [I0], [R0]
    t = np.linspace(0, T, passos + 1)

    for _ in range(passos):
        dE = beta * (S[-1] * I[-1] / N) * dt
        dI = sigma * E[-1] * dt
        dR = gamma * I[-1] * dt
        dS = omega * R[-1] * dt

        S.append(S[-1] - dE + dS)
        E.append(E[-1] + dE - dI)
        I.append(I[-1] + dI - dR)
        R.append(R[-1] + dR - dS)

    return np.array(I), t

# Modelo SEIRS (Equações Diferenciais)
def seirs_diff(t, y, beta, sigma, gamma, omega, N):
    S, E, I, R = y
    dS = -beta * (S * I / N) + omega * R
    dE = beta * (S * I / N) - sigma * E
    dI = sigma * E - gamma * I
    dR = gamma * I - omega * R
    return [dS, dE, dI, dR]

def solve_seirs(beta, sigma, gamma, omega, N, S0, E0, I0, R0, dt, T):
    y0 = [S0, E0, I0, R0]
    intervalo = [0, T]
    t_eval = np.arange(0, T + dt, dt)
    t_eval = t_eval[t_eval <= T]

    sol = solve_ivp(seirs_diff, intervalo, y0, args=(beta, sigma, gamma, omega, N), t_eval=t_eval)
    return sol.t, sol.y[2]

# Widgets Interativos
beta_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Beta')
sigma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Sigma')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Gamma')
omega_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='Omega')
N_slider = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='N')
E0_slider = widgets.IntSlider(min=1, max=500, step=1, value=10, description='E0')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=10, description='I0')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='R0')
T_slider = widgets.IntSlider(min=50, max=1000, step=50, value=365, description='T')
dt_disc_slider = widgets.FloatSlider(min=0.05, max=1.0, step=0.05, value=1.0, description='dt Discreto')
dt_cont_slider = widgets.FloatSlider(min=0.02, max=2.0, step=0.02, value=0.05, description='dt Contínuo')

def plot_seirs(beta, sigma, gamma, omega, N, E0, I0, R0, dt_disc, dt_cont, T):
    S0 = N - E0 - I0 - R0

    # Mede o tempo de CPU para o modelo com equações de diferenças
    t_ini = time.process_time()
    I_disc, t_disc = seirs_disc(S0, E0, I0, R0, beta, sigma, gamma, omega, N, dt_disc, T)
    cpu_time_disc = time.process_time() - t_ini

    # Mede o tempo de CPU para o modelo com equações diferenciais
    t_ini_diff = time.process_time()
    t_diff, I_diff = solve_seirs(beta, sigma, gamma, omega, N, S0, E0, I0, R0, dt_cont, T)
    cpu_time_diff = time.process_time() - t_ini_diff

    plt.figure(figsize=(10, 6), dpi=100)
    plt.plot(t_disc, I_disc, label='$I$ (Discreto)', color='red', linestyle='dashed')
    plt.plot(t_diff, I_diff, label='$I$ (Contínuo)', color='blue')
    plt.xlabel("$t$ (dias)")
    plt.ylabel("$I(t)$")
    plt.legend()
    plt.grid()
    plt.savefig("fig_seirs10.png", dpi=300)  # Salvar em alta resolução
    plt.show()

    # Exibe os tempos de CPU calculados
    print("Tempo de CPU (Discreto): {:.5f} s".format(cpu_time_disc))
    print("Tempo de CPU (Contínuo): {:.5f} s".format(cpu_time_diff))

# Cria a interface interativa
ui = widgets.VBox([
    beta_slider, sigma_slider, gamma_slider, omega_slider, N_slider,
    E0_slider, I0_slider, R0_slider, T_slider, dt_disc_slider, dt_cont_slider
])
out = widgets.interactive_output(plot_seirs, {
    'beta': beta_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'omega': omega_slider,
    'N': N_slider,
    'E0': E0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'T': T_slider,
    'dt_disc': dt_disc_slider,
    'dt_cont': dt_cont_slider
})
display(ui, out)


Output()